In [2]:
import time
import os
import glob
import uuid
import concurrent
import concurrent.futures
import requests
import json
import datetime
import soundfile as sf
import io
from pathlib import Path
import numpy as np
import torch
from einops import repeat
from torch import Tensor
from torch.nn import functional as F
from transformers import AutoTokenizer
from vocos import get_voco
from model.module import AudioBoxModule
from torchode.interface import solve_ivp
import torchaudio
import re
from tqdm import tqdm
import pandas as pd
from einops import rearrange

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
path = "./new-stage-2.ckpt"

model = AudioBoxModule.load_from_checkpoint(path).to(device)
model.train()
print("-")

/usr/local/lib/python3.10/dist-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: '/usr/local/lib/python3.10/dist-packages/torchvision/image.so: undefined symbol: _ZN3c1017RegisterOperatorsD1Ev'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


-


In [3]:
import pandas as pd
from data.dataset_0402 import AudioDataset
from torch.utils.data import DataLoader, random_split

datasets = AudioDataset(
    dataset_path = './test.csv',
    max_audio_len = 400,
    sampling_rate = 44100,
    max_txt_len = 127,
    channel = 2
)

dl = DataLoader(
    datasets,
    batch_size=4,
    num_workers=4,
    pin_memory=True,
    drop_last=True,
    shuffle=True,
    collate_fn=datasets.collate,
    prefetch_factor=4,
)

In [7]:
from config.train.config import TrainConfig
TrainConfig.__dict__

mappingproxy({'__module__': 'config.train.config',
              '__annotations__': {'acc': int,
               'batch_size': int,
               'base_path': str,
               'compile': bool,
               'early_stop': bool,
               'fast_dev_run': bool,
               'gradient_clip_val': float,
               'lr': float,
               'max_steps': int,
               'mlflow': bool,
               'num_workers': int,
               'optimizer': str,
               'resume_path': str | None,
               'scheduler': str,
               'precision': str,
               'project': str,
               'tracking_uri': str,
               'weight_average': bool},
              'acc': 1,
              'batch_size': 4,
              'base_path': '',
              'compile': False,
              'early_stop': False,
              'fast_dev_run': False,
              'gradient_clip_val': 0.2,
              'lr': 0.0002,
              'max_steps': 1000000,
              'mlflo

In [13]:
import torch

lr = 0.0002
epochs = 10
optimizer_type = "Adam"
scheduler_type = "linear_warmup_decay"
voco_type = 'oobleck'

In [10]:
from torch.optim.lr_scheduler import LinearLR, SequentialLR
from torch.optim.adam import Adam

max_steps = 50000

optimizer = Adam(model.parameters(), lr=lr)

warmup_scheduler = LinearLR(
    optimizer, 
    start_factor=1 / 5000, 
    end_factor=1.0, 
    total_iters=5000
)
decay_scheduler = LinearLR(
    optimizer,
    start_factor=1.0,
    end_factor=0.0001,
    total_iters=max_steps-5000,
)
scheduler = SequentialLR(
    optimizer,
    schedulers=[warmup_scheduler, decay_scheduler],
    milestones=[5000],
)

In [12]:
for epoch in range(epochs):
    train_loss = 0
    val_loss = 0
    
    model.train()
    for batch in dl:
        optimizer.zero_grad()
        
        latent, latent_mask, text_input_ids, text_attention_mask = batch
        latent = latent.to(device)
        latent_mask = latent_mask.to(device)
        text_input_ids = text_input_ids.to(device)
        text_attention_mask = text_attention_mask.to(device)
    
        loss = model(
            audio_enc=latent,
            audio_mask=latent_mask,
            phoneme=text_input_ids,
            phoneme_mask=text_attention_mask
        )
        print(loss)
        loss.backward()
        optimizer.step()
        scheduler.step()

    print("validate")
    model.eval()
    for batch in dl:
        optimizer.zero_grad()
        
        latent, latent_mask, text_input_ids, text_attention_mask = batch
        latent = latent.to(device)
        latent_mask = latent_mask.to(device)
        text_input_ids = text_input_ids.to(device)
        text_attention_mask = text_attention_mask.to(device)
    
        loss = model(
            audio_enc=latent,
            audio_mask=latent_mask,
            phoneme=text_input_ids,
            phoneme_mask=text_attention_mask
        )
        val_loss += loss.item()
        
    

tensor(0.9803, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(0.8116, device='cuda:0', grad_fn=<MeanBackward0>)
